In [1]:
import tsKNEE 
import scanpy as sc
import leidenalg

In [2]:
adata = sc.read_h5ad("../test_data/test_data1.h5ad")
sc.pp.neighbors(adata, use_rep='X')
sc.tl.leiden(adata)

In [3]:
print('Run Time Benchmark of Clustering')
print('tSNE')
%time sc.tl.tsne(adata)
print('tsKNEE')
%time x = tsKNEE.tsKNEE(adata)

Run Time Benchmark of Clustering
tSNE
         Falling back to preprocessing with `sc.pp.pca` and default params.
CPU times: user 1.82 s, sys: 69.9 ms, total: 1.89 s
Wall time: 272 ms
tsKNEE
CPU times: user 617 ms, sys: 6.41 ms, total: 624 ms
Wall time: 601 ms


In [4]:
import os
pathToMem = os.path.abspath("memusg/memusg")
os.environ['PATH'] += os.pathsep + os.path.dirname(pathToMem)

In [5]:
print('Run Time Benchmark of Plotting')
print('tSNE')
!memusg python -c "import scanpy as sc; import your_data_loading_module; adata = your_data_loading_module.load_adata(); sc.pl.tsne(adata)"
print('tsKNEE')
!memusg python -c "import tsKNEE; import your_data_loading_module; adata = your_data_loading_module.load_adata(); tsKNEE.tsKNEE_plot(adata)"

Run Time Benchmark of Plotting
tSNE
/bin/bash: memusg: command not found
tsKNEE
/bin/bash: memusg: command not found


In [6]:
print('Run Time Benchmark of Plotting')

# tSNE
print('tSNE')
!{path_to_mem}/memusg python -c "import scanpy as sc; import your_data_loading_module; adata = your_data_loading_module.load_adata(); sc.pl.tsne(adata)"

# tsKNEE
print('tsKNEE')
!{path_to_mem}/memusg python -c "import tsKNEE; import your_data_loading_module; adata = your_data_loading_module.load_adata(); tsKNEE.tsKNEE_plot(adata)"

Run Time Benchmark of Plotting
tSNE
/bin/bash: {path_to_mem}/memusg: No such file or directory
tsKNEE
/bin/bash: {path_to_mem}/memusg: No such file or directory


In [2]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances

def continuity_score(X_high, X_low, n_neighbors=5):
    """
    Compute the continuity score of the low-dimensional embedding.

    Parameters:
    X_high : array-like, shape (n_samples, n_features)
        High-dimensional data.
    X_low : array-like, shape (n_samples, n_components)
        Low-dimensional embedding.
    n_neighbors : int, optional, default: 5
        Number of nearest neighbors to consider.

    Returns:
    continuity : float
        Continuity score.
    """
    n_samples = X_high.shape[0]

    # Find nearest neighbors in high-dimensional space
    nbrs_high = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X_high)
    distances_high, indices_high = nbrs_high.kneighbors(X_high)

    # Find nearest neighbors in low-dimensional space
    nbrs_low = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X_low)
    distances_low, indices_low = nbrs_low.kneighbors(X_low)

    # Remove self-indices (first column)
    indices_high = indices_high[:, 1:]
    indices_low = indices_low[:, 1:]

    # Compute continuity score
    continuity = 0
    for i in range(n_samples):
        intersection = len(set(indices_high[i]).intersection(set(indices_low[i])))
        continuity += intersection / n_neighbors

    continuity /= n_samples
    return continuity

In [10]:
# Load data
my_adata = sc.read_h5ad("../small_test_data/small_test_data1.h5ad")
sc.pp.neighbors(my_adata)
sc.tl.leiden(my_adata)    

tsne_adata = sc.read_h5ad("../small_test_data/small_test_data1.h5ad")
sc.pp.neighbors(tsne_adata)
sc.tl.leiden(tsne_adata) 

# Compute t-SNE
sc.tl.tsne(tsne_adata)
tsKNEE.tsKNEE(my_adata)

# Calculate continuity
my_continuity = continuity_score(my_adata.X, my_adata.obsm['X_tsne'], n_neighbors=5)
print(f"tsKNEE continuity: {my_continuity}")

tsne_continuity = continuity_score(tsne_adata.X, tsne_adata.obsm['X_tsne'], n_neighbors=5)
print(f"tsne continuity: {tsne_continuity}")

         Falling back to preprocessing with `sc.pp.pca` and default params.


         Falling back to preprocessing with `sc.pp.pca` and default params.
tsKNEE continuity: 0.6057971014492755
tsne continuity: 0.2840579710144926
